In [20]:
using BenchmarkTools
using ViewerGL, LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors, DataStructures, SparseArrays
using SuiteSparseGraphBLAS, GraphBLASInterface
using SparseMM
using LocalCongruence

In [22]:
GL = ViewerGL
Lar = LinearAlgebraicRepresentation
LC = LocalCongruence
GrB_init(GrB_NONBLOCKING)

GrB_SUCCESS::GrB_Info = 0

## ESEMPIO 1: Julia Native Sparse Matrix

In [23]:
G = [
	0.5310492999999998 1.0146684 0.3477716 0.8313907882395298 0.6061046999999998 1.0897237999999998 0.42282699999999984 0.9064461979373597 0.5310493 1.0146684000000001 0.6061047 1.0897237772434623 0.3477716 0.8313908 0.422827 0.9064462 0.5310493 0.34777160000000007 0.6061047 0.4228270000000002 1.0146684 0.8313908 1.0897238 0.9064461675456482;
	0.8659989999999999 0.6827212999999999 0.5268921 0.3436144447063971 1.2188832999999994 1.0356056999999999 0.8797763999999998 0.6964987903021808 0.8659989999999999 0.6827213 1.2188833 1.035605657895053 0.5268921 0.3436145 0.8797764000000001 0.6964988 0.8659989999999999 0.5268920999999999 1.2188833 0.8797764 0.6827213 0.3436145 1.0356057 0.6964988122992563;
	0.14191280000000003 0.2169682 0.4947971000000001 0.5698524407571428 0.5200012 0.5950565999999999 0.8728855 0.9479408896095312 0.14191279999999987 0.21696819999999994 0.5200011999999999 0.5950566438156151 0.4947971 0.5698525000000001 0.8728855 0.9479409 0.14191280000000006 0.4947971000000001 0.5200012000000002 0.8728855000000001 0.21696819999999994 0.5698525000000001 0.5950565999999999 0.9479408949632379
];

In [24]:
W = convert(Lar.Points, G')

24×3 Matrix{Float64}:
 0.531049  0.865999  0.141913
 1.01467   0.682721  0.216968
 0.347772  0.526892  0.494797
 0.831391  0.343614  0.569852
 0.606105  1.21888   0.520001
 1.08972   1.03561   0.595057
 0.422827  0.879776  0.872885
 0.906446  0.696499  0.947941
 0.531049  0.865999  0.141913
 1.01467   0.682721  0.216968
 0.606105  1.21888   0.520001
 1.08972   1.03561   0.595057
 0.347772  0.526892  0.494797
 0.831391  0.343614  0.569853
 0.422827  0.879776  0.872885
 0.906446  0.696499  0.947941
 0.531049  0.865999  0.141913
 0.347772  0.526892  0.494797
 0.606105  1.21888   0.520001
 0.422827  0.879776  0.872886
 1.01467   0.682721  0.216968
 0.831391  0.343614  0.569853
 1.08972   1.03561   0.595057
 0.906446  0.696499  0.947941

In [25]:
Delta_0 = SparseArrays.sparse((
	[1, 3, 1, 4, 2, 3, 2, 4, 5, 7, 5, 8, 6, 7, 6, 8, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 17, 19, 17, 20, 18, 19, 18, 20, 21, 23, 21, 24, 22, 23, 22, 24],
	[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24],
	Int8[-1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1]
)...);

In [26]:
Delta_1 = SparseArrays.sparse((
	[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
	[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
	Int8[1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1]
)...);
T = [Delta_0, Delta_1];

In [28]:
@benchmark LC.chainCongruenceSM(G, T)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  104.199 μs …   7.836 ms  ┊ GC (min … max): 0.00% … 98.20%
 Time  (median):     111.739 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   123.565 μs ± 257.329 μs  ┊ GC (mean ± σ):  8.02% ±  3.79%

   ▂▁▂▃██▂                                                       
  ▃███████▇▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂ ▃
  104 μs           Histogram: frequency by time          184 μs <

 Memory estimate: 84.20 KiB, allocs estimate: 1303.

## ESEMPIO 1: GraphBLAS based

In [29]:
Delta_0_GB = SparseMM.sm2gbm(Delta_0)
Delta_1_GB = SparseMM.sm2gbm(Delta_1)
T_GB = [Delta_0_GB, Delta_1_GB]

2-element Vector{GrB_Matrix{Int8}}:
 
 

GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}

In [30]:
@benchmark LC.chainCongruenceGB(W, T_GB)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  114.358 μs …   5.765 ms  ┊ GC (min … max): 0.00% … 97.42%
 Time  (median):     117.994 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   123.385 μs ± 125.496 μs  ┊ GC (mean ± σ):  2.26% ±  2.18%

    ▁█▇▁                                                         
  ▁▃████▆▆▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  114 μs           Histogram: frequency by time          157 μs <

 Memory estimate: 27.52 KiB, allocs estimate: 477.

## ESEMPIO 2: Julia Native Sparse Matrix

In [31]:
G = [0.4851363000000001 0.5099412000000001 0.4604538000000001 0.4852586932051473 0.5099412 0.5347462 0.4852588 0.5100638087892613 0.5347461999999997 0.5595511999999998 0.5100636999999998 0.5348687087555339 0.46045379999999997 0.48525879999999993 0.4357712999999999 0.46057630875553396 0.4852587999999998 0.5100636999999998 0.4605762999999998 0.485381191210697 0.5100636999999999 0.5348686999999999 0.4853812999999999 0.510186289283825 0.43577130000000003 0.46057630000000005 0.41108890000000003 0.4358938892838253 0.46057630000000005 0.48538130000000007 0.43589380000000005 0.4606988195054789 0.48538129999999996 0.5101861999999999 0.46069879999999996 0.4855036999999999 0.7325833 0.7573883 0.7079008 0.732705808755534 0.7573882999999998 0.7821931999999998 0.7327057999999997 0.7575106892500558 0.7821932000000001 0.8069982000000001 0.7575107000000001 0.7823156912444693 0.7079008 0.7327058 0.6832184 0.7080233892838251 0.7327057999999999 0.7575106999999999 0.7080232999999999 0.7328282127105769 0.7575107 0.7823156999999998 0.7328283 0.7576333067948982 0.6832184000000002 0.7080233000000002 0.6585359000000002 0.6833408000000002 0.7080233000000001 0.7328283000000001 0.6833408000000001 0.7081457892500528 0.7328283 0.7576332 0.7081458 0.7329506932051473 0.9800303 1.0048353 0.9553478 0.9801527912444694 1.0048353000000003 1.0296402000000002 0.9801528000000003 1.004957712710577 1.0296402 1.0544452 1.0049578 1.029762806794898 0.9553478000000003 0.9801528000000004 0.9306654000000003 0.9554703980393594 0.9801528 1.0049578 0.9554703 0.9802752892500525 1.0049577999999997 1.0297626999999998 0.9802752999999997 1.0050801932051472 0.9306654 0.9554703 0.9059829 0.9307878019606395 0.9554702999999996 0.9802752999999995 0.9307878999999994 0.9555929175448041 0.9802753000000002 1.0050803000000001 0.9555928000000002 0.9803978000000003 1.2274773000000003 1.2522823000000003 1.2027949000000002 1.2275999067948984 1.2522823 1.2770873 1.2275998 1.2524047892500525 1.2770873 1.3018922 1.2524048 1.2772096932051473 1.2027949 1.2275998 1.1781124 1.2029172932051473 1.2275998000000001 1.2524048000000003 1.2029174000000002 1.227722408789262 1.2524047999999997 1.2772097999999996 1.2277222999999995 1.2525273087555335 1.1781123999999998 1.2029173999999998 1.1534298999999997 1.1782348999999996 1.2029174 1.2277223 1.1782349 1.2030397892500562 1.2277222999999997 1.2525272999999997 1.2030397999999998 1.227844791244469 0.4851363000000001 0.5099412000000001 0.7325833000000002 0.7573882979522348 0.5099411999999999 0.5347462 0.7573883000000001 0.7821931912058342 0.5347461999999998 0.5595511999999999 0.7821932 0.806998201072288 0.46045379999999997 0.4852587999999999 0.7079008 0.7327058010722881 0.4852588000000001 0.5100637 0.7327058000000001 0.7575107097696032 0.5100637000000002 0.5348687000000001 0.7575107000000001 0.7823156989277071 0.43577129999999986 0.4605762999999999 0.6832183999999999 0.7080232999031792 0.4605762999999998 0.48538129999999985 0.7080232999999998 0.7328282902303588 0.4853812999999999 0.5101861999999999 0.7328283 0.7576332010722877 0.4110888999999999 0.4358938 0.6585359 0.6833408000000001 0.4358937999999999 0.4606987999999999 0.6833408 0.7081458086973149 0.4606988000000001 0.48550370000000015 0.7081458000000003 0.7329507990245284 0.7325833000000002 0.7573883000000001 0.9800303000000001 1.0048353010722884 0.7573882999999998 0.7821931999999999 1.0048352999999999 1.0296402097696031 0.7821932000000001 0.8069982 1.0296402 1.054445198927707 0.7079008 0.7327058000000001 0.9553478 0.9801527989277072 0.7327058 0.7575107 0.9801528 1.0049577892548855 0.7575106999999999 0.7823156999999996 1.0049578 1.0297627020477507 0.6832184 0.7080233000000002 0.9306654000000002 0.9554703000000001 0.7080233 0.7328282999999999 0.9554703 0.9802753097696029 0.7328283 0.7576332 0.9802753000000002 1.0050802979522349 0.6585359000000001 0.6833408000000001 0.9059829 0.9307878990245285 0.6833408000000002 0.7081458 0.9307879000000001 0.9555927922781232 0.7081458000000002 0.7329508000000002 0.9555928 0.9803978 0.9800302999999997 1.0048352999999999 1.2274773 1.252282298927707 1.0048353 1.0296402 1.2522822999999996 1.2770872892548857 1.0296401999999998 1.0544451999999997 1.2770872999999998 1.3018922020477506 0.9553478000000001 0.9801528000000002 1.2027949000000002 1.2275998020477508 0.9801527999999998 1.0049577999999997 1.2275997999999997 1.2524048097696028 1.0049578 1.0297626999999998 1.2524048000000003 1.277209797952235 0.9306653999999999 0.9554703000000001 1.1781123999999998 1.2029173990245283 0.9554702999999999 0.9802752999999997 1.2029173999999996 1.227722291205834 0.9802753 1.0050803 1.2277223 1.2525273010722882 0.9059828999999999 0.9307878999999998 1.1534299 1.1782348999999996 0.9307879000000002 0.9555927999999999 1.1782349 1.2030398086973157 0.9555928 0.9803977999999999 1.2030398 1.2278448 0.48513629999999996 0.4604538000000001 0.7325833 0.7079008088474859 0.5099411999999998 0.48525879999999993 0.7573882999999999 0.7327057901770416 0.5347461999999998 0.5100637 0.7821931999999999 0.7575106911524817 0.5595512 0.5348686999999999 0.8069982 0.7823157108081016 0.4604538 0.4357713 0.7079008 0.6832183901673042 0.48525880000000005 0.4605763 0.7327058000000001 0.7080233108081018 0.5100637000000001 0.48538130000000007 0.7575107 0.7328283088474857 0.5348687000000001 0.5101862 0.7823157000000001 0.7576331891918608 0.43577129999999986 0.4110888999999999 0.6832183999999999 0.6585359000048607 0.4605762999999998 0.43589379999999994 0.7080232999999998 0.6833407891918606 0.4853812999999999 0.46069879999999985 0.7328283 0.7081458009803131 0.5101862 0.4855037 0.7576332 0.7329508098229311 0.7325833 0.7079007999999998 0.9800303000000001 0.9553477911524815 0.7573883 0.7327058000000001 1.0048353000000003 0.9801528108081017 0.7821932000000001 0.7575107 1.0296402 1.0049578078623143 0.8069982000000001 0.7823157000000001 1.0544452000000002 1.0297626891918608 0.7079007999999999 0.6832183999999999 0.9553477999999999 0.9306653990196898 0.7327057999999999 0.7080232999999998 0.9801527999999999 0.9554702891918605 0.7575107000000001 0.7328283000000001 1.0049578000000001 0.9802752921376624 0.7823157000000001 0.7576332 1.0297627 1.0050803098229308 0.6832184 0.6585359 0.9306654000000003 0.9059829108081019 0.7080233 0.6833407999999999 0.9554703 0.9307879098229311 0.7328283 0.7081458000000002 0.9802753000000002 0.9555927990196903 0.7576332 0.7329508 1.0050803000000001 0.9803977901770418 0.9800303000000001 0.9553478000000003 1.2274773000000005 1.202794907862315 1.0048353 0.9801527999999999 1.2522822999999998 1.2275997891918604 1.0296401999999998 1.0049577999999997 1.2770872999999998 1.2524047921376624 1.0544452 1.0297627 1.3018922 1.277209809822931 0.9553478 0.9306654000000001 1.2027949 1.1781124019654912 0.9801527999999999 0.9554703 1.2275997999999997 1.2029174098229312 1.0049578 0.9802753 1.2524048000000003 1.2277223088474856 1.0297626999999998 1.0050802999999997 1.2772097999999998 1.2525272901770415 0.9306654 0.9059829000000001 1.1781124 1.1534298891918606 0.9554703000000002 0.9307879000000004 1.2029174 1.178234890177042 0.9802753 0.9555927999999999 1.2277223 1.2030398009803132 1.0050803 0.9803978 1.2525273 1.2278448108081017;
0.9680746999999998 0.9433922 1.2152774 1.190594888290028 0.9433921999999999 0.9187098 1.1905949 1.165912411928616 0.9187097999999998 0.8940273 1.1659124 1.1412299009600377 1.2152773999999995 1.1905948999999998 1.4624799999999996 1.4377975009600377 1.1905948999999998 1.1659123999999996 1.4377974999999996 1.4131150880713321 1.1659123999999998 1.1412299 1.4131150999999997 1.38843260978987 1.4624800000000002 1.4377975000000005 1.7096827 1.6850002097898702 1.4377975000000005 1.4131151000000002 1.6850002000000004 1.6603177021387612 1.4131150999999995 1.3884325999999998 1.6603176999999998 1.6356351999999998 0.9952071000000002 0.9705246000000001 1.2424097000000005 1.217727200960038 0.9705245999999998 0.9458421 1.2177271999999997 1.1930447988212762 0.9458420999999999 0.9211596 1.1930448 1.1683622990399625 1.2424097 1.2177272000000001 1.4896124000000002 1.46492990978987 1.2177271999999997 1.1930447999999998 1.4649298999999998 1.4402473904287851 1.1930448 1.1683623 1.4402474000000003 1.4155649117099405 1.4896124000000002 1.4649299000000005 1.7368150000000007 1.7121325000000003 1.4649298999999998 1.4402473999999998 1.7121324999999994 1.687450098821275 1.4402474000000003 1.4155649000000001 1.6874501000000002 1.662767588290028 1.0223393999999997 0.9976568999999998 1.2695420999999998 1.2448595990399625 0.9976569000000004 0.9729745000000004 1.2448596000000005 1.2201770904287856 0.9729745000000001 0.9482920000000001 1.2201771 1.1954946117099403 1.2695421 1.2448596 1.5167447 1.492062210749909 1.2448596000000003 1.2201771000000001 1.4920622000000003 1.4673797988212758 1.2201771 1.1954946000000002 1.4673798 1.4426972882900277 1.5167447000000003 1.4920622 1.7639474000000002 1.7392648892500637 1.4920621999999997 1.4673797999999998 1.7392648999999998 1.7145824128886509 1.4673798 1.4426973 1.7145824000000003 1.6898998999999997 1.0494717999999998 1.0247893 1.2966744000000001 1.2719919117099405 1.0247893 1.0001067999999997 1.2719919 1.2473094988212754 1.0001068 0.9754243 1.2473095 1.2226269882900276 1.2966744 1.2719919000000002 1.5438771 1.5191945882900277 1.2719919 1.2473095000000003 1.5191946 1.4945121119286158 1.2473094999999998 1.2226269999999997 1.4945120999999997 1.4698296009600378 1.5438771 1.5191945999999998 1.7910796999999998 1.7663972000000001 1.5191945999999998 1.4945120999999997 1.7663971999999997 1.7417147988212762 1.4945121000000001 1.4698296000000002 1.7417148000000002 1.7170322990399627 0.9680747000000001 0.9433922000000001 0.9952071000000001 0.9705246205089642 0.9433921999999998 0.9187097999999998 0.9705245999999997 0.9458421880761753 0.9187098 0.8940273 0.9458421 0.9211595892606733 1.2152773999999997 1.1905948999999998 1.2424096999999998 1.2177271892606731 1.1905949 1.1659124 1.2177272 1.1930447021541872 1.1659123999999996 1.1412298999999995 1.1930447999999996 1.1683623107393253 1.4624800000000002 1.4377975000000003 1.4896124000000002 1.464929900969688 1.4377974999999998 1.4131150999999997 1.4649298999999998 1.4402474978458117 1.4131150999999997 1.3884325999999998 1.4402473999999998 1.4155648892606771 1.7096827 1.6850002 1.736815 1.7121324999999998 1.6850002000000002 1.6603177000000002 1.7121325000000003 1.6874500128935104 1.6603177000000002 1.6356352000000003 1.6874501000000002 1.6627676097696376 0.9952071000000001 0.9705246000000001 1.0223394000000001 0.9976568892606733 0.9705246000000001 0.9458421000000001 0.9976569000000001 0.9729744021541875 0.9458421000000001 0.9211596000000001 0.9729745000000001 0.9482920107393258 1.2424096999999998 1.2177271999999997 1.2695420999999998 1.2448596107393255 1.2177272000000001 1.1930448000000002 1.2448596000000003 1.220177207615455 1.1930448 1.1683623 1.2201771 1.1954945794910787 1.4896123999999995 1.4649298999999996 1.5167446999999996 1.4920621999999997 1.4649299 1.4402474 1.4920622 1.4673797021541881 1.4402473999999998 1.4155648999999997 1.4673797999999998 1.4426973205089635 1.736815 1.7121325 1.7639474 1.7392649097696373 1.7121325000000003 1.6874501000000002 1.7392649000000002 1.7145824773368845 1.6874501000000002 1.6627676000000002 1.7145824000000003 1.6898999000000003 1.0223394 0.9976569 1.0494718 1.024789310739326 0.9976568999999997 0.9729744999999997 1.0247892999999997 1.000106907615454 0.9729744999999999 0.9482919999999999 1.0001067999999997 0.9754242794910788 1.2695421 1.2448596 1.2966744 1.2719918794910787 1.2448595999999998 1.2201770999999997 1.2719918999999997 1.2473094021541877 1.2201771 1.1954946 1.2473095 1.2226270205089642 1.5167446999999998 1.4920621999999997 1.5438770999999998 1.519194609769637 1.4920621999999995 1.4673797999999996 1.5191945999999996 1.4945121880761751 1.4673797999999998 1.4426972999999998 1.4945120999999997 1.469829589260673 1.7639474000000002 1.7392649000000002 1.7910797000000005 1.7663972000000003 1.7392649 1.7145824 1.7663972 1.7417147128935053 1.7145824 1.6898999 1.7417148 1.7170322999999998 0.9680746999999997 1.2152774 0.9952071 1.242409691196175 0.9433921999999998 1.1905948999999998 0.9705245999999998 1.217727209774413 0.9187097999999998 1.1659124 0.9458420999999999 1.1930448088037933 0.8940273000000002 1.1412299 0.9211596 1.1683622892452266 1.2152773999999997 1.4624799999999998 1.2424097 1.4896124097841024 1.1905948999999998 1.4377974999999998 1.2177271999999995 1.4649298892452263 1.1659123999999998 1.4131150999999997 1.1930448 1.4402473911961726 1.1412299 1.3884326000000002 1.1683623000000003 1.4155649107547168 1.4624800000000002 1.7096827 1.4896124000000004 1.7368149999951634 1.4377975 1.6850002 1.4649298999999996 1.7121325107547167 1.4131150999999995 1.6603176999999998 1.4402473999999998 1.6874500990245278 1.3884326000000002 1.6356352000000003 1.4155649 1.6627675902255363 0.9952070999999999 1.2424097 1.0223393999999997 1.2695421088037933 0.9705245999999998 1.2177271999999997 0.9976568999999998 1.2448595892452265 0.9458420999999999 1.1930447999999998 0.9729744999999999 1.220177092176485 0.9211596 1.1683623000000003 0.948292 1.1954946107547169 1.2424096999999998 1.4896124 1.2695421 1.5167447009754724 1.2177271999999997 1.4649298999999998 1.2448596 1.4920622107547166 1.1930448 1.4402474000000003 1.2201771 1.4673798078234879 1.1683623 1.4155649 1.1954946 1.442697290225536 1.4896123999999997 1.736815 1.5167447 1.7639473892452262 1.4649299000000002 1.7121324999999998 1.4920622000000001 1.7392648902255363 1.4402474000000003 1.6874501000000002 1.4673798 1.714582400975472 1.4155649 1.6627676 1.4426973000000003 1.6898999097744127 1.0223394 1.2695421 1.0494718000000003 1.296674392176485 0.9976569000000002 1.2448596000000003 1.0247893 1.2719919107547168 0.9729744999999999 1.2201770999999997 1.0001067999999997 1.247309507823488 0.948292 1.1954946 0.9754242999999999 1.2226269902255362 1.2695421 1.5167447 1.2966744 1.5438770980442142 1.2448596 1.4920622000000001 1.2719919000000002 1.5191945902255362 1.2201771 1.4673798 1.2473095 1.4945120911961751 1.1954946 1.4426973 1.2226269999999997 1.469829609774413 1.5167447000000005 1.7639474000000006 1.5438771000000002 1.7910797107547172 1.4920622000000001 1.7392649000000002 1.5191946 1.7663972097744132 1.4673798000000002 1.7145824000000005 1.4945121000000001 1.741714799024528 1.4426972999999998 1.6898999 1.4698296 1.7170322892452263;
0.9097499999999998 1.1571971 0.9368824 1.1843294095374093 1.1571971 1.4046441 1.1843294000000002 1.4317763902844214 1.4046441 1.6520911 1.4317764 1.6792234992180755 0.9368824000000001 1.1843294 0.9640147000000001 1.2114617992180752 1.1843293999999998 1.4317763999999997 1.2114617999999997 1.4589088097155285 1.4317763999999997 1.6792234999999998 1.4589087999999997 1.706355792026389 0.9640147 1.2114618 0.9911471 1.2385940920263896 1.2114618000000001 1.4589088000000001 1.2385941 1.4860411982580366 1.4589087999999997 1.7063557999999996 1.4860411999999998 1.7334881999999996 0.8876515 1.1350985 0.9147838 1.1622308992180752 1.1350985 1.3825455000000002 1.1622309 1.4096779009600338 1.3825455000000004 1.6299926000000005 1.4096779000000002 1.657124900781924 0.9147838000000001 1.1622308999999997 0.9419162 1.1893631920263896 1.1622309 1.4096779 1.1893632 1.4368102077954668 1.4096779 1.6571248999999997 1.4368102 1.684257290462536 0.9419161999999998 1.1893631999999998 0.9690485999999999 1.2164955999999998 1.1893631999999998 1.4368101999999998 1.2164955999999996 1.463942600960038 1.4368102 1.6842573 1.4639426 1.711389609537409 0.8655529 1.1129999999999998 0.8926852999999999 1.1401323007819242 1.113 1.3604470000000004 1.1401323000000003 1.387579307795467 1.360447 1.6078939999999997 1.3875793 1.635026390462536 0.8926853 1.1401323 0.9198177 1.1672646912444617 1.1401323000000003 1.3875793000000003 1.1672647 1.4147117009600385 1.3875792999999998 1.6350263999999999 1.4147117 1.662158709537409 0.9198177 1.1672646999999998 0.94695 1.1943970087554958 1.1672647 1.4147117000000002 1.194397 1.4418440895024953 1.4147117000000002 1.6621587 1.4418441000000002 1.6892911 0.8434544 1.0909014000000001 0.8705867 1.1180337904625361 1.0909014 1.3383484 1.1180337999999999 1.365480800960038 1.3383484 1.5857955 1.3654807999999998 1.612927809537409 0.8705866999999998 1.1180337999999999 0.8977190999999999 1.145166109537409 1.1180338 1.3654808000000003 1.1451661 1.3926130902844216 1.3654808 1.6129277999999998 1.3926131 1.640060199218075 0.8977191000000001 1.1451661000000002 0.9248515 1.1722985000000001 1.1451661 1.3926131 1.1722984999999997 1.4197455009600337 1.3926131 1.6400602 1.4197455000000003 1.6671925007819244 0.9097499999999998 1.1571971 0.8876514999999999 1.135098502251015 1.1571970999999999 1.4046440999999998 1.1350985 1.382545509667002 1.4046441 1.6520911 1.3825455 1.6299925988212767 0.9368824000000001 1.1843294 0.9147838000000001 1.1622308988212766 1.1843293999999998 1.4317763999999997 1.1622308999999997 1.4096778892606765 1.4317764 1.6792235 1.4096778999999997 1.657124901178723 0.9640147 1.2114618 0.9419162000000001 1.1893632001064307 1.2114617999999997 1.4589087999999997 1.1893631999999998 1.4368102107392913 1.4589088 1.7063557999999999 1.4368101999999998 1.6842572988212776 0.9911471000000001 1.2385941 0.9690486000000001 1.2164956 1.2385940999999998 1.4860411999999998 1.2164956 1.4639425904393994 1.4860412000000003 1.7334882000000003 1.4639426000000004 1.7113896010722924 0.8876515 1.1350985 0.8655528999999997 1.1129999988212766 1.1350984999999998 1.3825455 1.1129999999999998 1.3604469892606765 1.3825455000000004 1.6299926000000005 1.3604470000000002 1.6078940011787235 0.9147838 1.1622308999999997 0.8926852999999999 1.1401323011787232 1.1622309 1.4096779 1.1401323 1.3875793118115767 1.4096778999999997 1.6571248999999995 1.3875792999999998 1.6350263977489887 0.9419162 1.1893632 0.9198177000000001 1.1672647 1.1893632 1.4368102 1.1672647 1.414711689260673 1.4368102 1.6842573 1.4147117 1.6621587022510147 0.9690486 1.2164956 0.9469500000000001 1.1943970010722924 1.2164956 1.4639426 1.194397 1.4418441084882867 1.4639426000000002 1.7113896 1.4418441000000002 1.6892911 0.8655528999999998 1.1129999999999995 0.8434543999999997 1.0909014011787228 1.1129999999999998 1.360447 1.0909013999999997 1.3383484118115767 1.360447 1.6078939999999997 1.3383484 1.5857954977489888 0.8926853 1.1401323 0.8705867 1.118033797748989 1.1401323 1.3875793 1.1180337999999999 1.3654807892606728 1.3875792999999998 1.6350263999999999 1.3654807999999998 1.612927802251015 0.9198177000000001 1.1672647 0.8977191000000001 1.1451661010722924 1.1672646999999998 1.4147116999999998 1.1451660999999997 1.392613109667002 1.4147117 1.6621587 1.3926131 1.6400601988212764 0.94695 1.194397 0.9248514999999998 1.1722985 1.194397 1.4418441 1.1722985 1.4197454904394027 1.4418441000000002 1.6892911 1.4197455000000003 1.6671925 0.9097499999999997 0.9368823999999997 0.8876514999999998 0.9147838882600601 1.1571970999999999 1.1843293999999998 1.1350984999999998 1.1622308020092107 1.4046441 1.4317764 1.3825455 1.4096778117398994 1.6520911 1.6792235 1.6299926 1.6571250078186572 0.9368824 0.9640147 0.9147838 0.9419161019120725 1.1843294 1.2114618 1.1622309 1.1893633078186572 1.4317764 1.4589088 1.4096779 1.4368102882600555 1.6792235 1.7063558 1.6571249 1.6842571921813883 0.9640147 0.9911471 0.9419162 0.9690486000484911 1.2114617999999997 1.2385940999999998 1.1893631999999998 1.2164954921813882 1.4589088 1.4860412 1.4368102 1.4639426097792962 1.7063558000000003 1.7334882000000003 1.6842573000000003 1.7113896979908656 0.8876515 0.9147838 0.8655529 0.8926852117398995 1.1350984999999998 1.1622308999999997 1.1129999999999998 1.140132407818657 1.3825455000000002 1.4096779000000002 1.3604470000000002 1.3875793784322652 1.6299926000000002 1.6571249000000001 1.6078940000000002 1.6350262921813883 0.9147838 0.9419162 0.8926853 0.9198176902206997 1.1622308999999997 1.1893631999999998 1.1401322999999999 1.1672645921813882 1.4096779000000002 1.4368102000000003 1.3875793000000003 1.414711621567728 1.6571248999999995 1.6842572999999996 1.6350263999999997 1.6621587979908652 0.9419162 0.9690486 0.9198177 0.9469501078186573 1.1893632 1.2164956 1.1672647 1.1943970979908654 1.4368102000000003 1.4639426000000002 1.4147117000000002 1.4418440902207037 1.6842573 1.7113896 1.6621587 1.689291002009211 0.8655529000000001 0.8926853000000001 0.8434544000000002 0.870586778432265 1.113 1.1401323 1.0909014 1.1180336921813883 1.3604469999999997 1.3875792999999998 1.3383483999999999 1.3654807215677276 1.607894 1.6350264 1.5857955 1.6129278979908654 0.8926853 0.9198177 0.8705867 0.8977191196071274 1.1401323 1.1672647 1.1180338 1.1451661979908654 1.3875793 1.4147117 1.3654808 1.3926131882600603 1.6350264 1.6621587 1.6129278 1.6400601020092112 0.9198177000000002 0.9469500000000002 0.8977191000000002 0.9248513921813886 1.1672647000000005 1.1943970000000004 1.1451661000000004 1.1722984020092113 1.4147117000000002 1.4418441000000002 1.3926131000000002 1.4197455097792961 1.6621587000000002 1.6892911000000002 1.6400602000000002 1.6671926078186574];

W = convert(Lar.Points, G')

432×3 Matrix{Float64}:
 0.485136  0.968075  0.90975
 0.509941  0.943392  1.1572
 0.460454  1.21528   0.936882
 0.485259  1.19059   1.18433
 0.509941  0.943392  1.1572
 0.534746  0.91871   1.40464
 0.485259  1.19059   1.18433
 0.510064  1.16591   1.43178
 0.534746  0.91871   1.40464
 0.559551  0.894027  1.65209
 0.510064  1.16591   1.43178
 0.534869  1.14123   1.67922
 0.460454  1.21528   0.936882
 ⋮                   
 0.95547   1.49206   1.16726
 0.930788  1.73926   1.1944
 1.20292   1.51919   1.14517
 1.17823   1.7664    1.1723
 0.980275  1.46738   1.41471
 0.955593  1.71458   1.44184
 1.22772   1.49451   1.39261
 1.20304   1.74171   1.41975
 1.00508   1.4427    1.66216
 0.980398  1.6899    1.68929
 1.25253   1.46983   1.64006
 1.22784   1.71703   1.66719

In [32]:
Delta_0 = SparseArrays.sparse(([1, 3, 1, 4, 2, 3, 2, 4, 5, 7, 5, 8, 6, 7, 6, 8, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 17, 19, 17, 20, 18, 19, 18, 20, 21, 23, 21, 24, 22, 23, 22, 24, 25, 27, 25, 28, 26, 27, 26, 28, 29, 31, 29, 32, 30, 31, 30, 32, 33, 35, 33, 36, 34, 35, 34, 36, 37, 39, 37, 40, 38, 39, 38, 40, 41, 43, 41, 44, 42, 43, 42, 44, 45, 47, 45, 48, 46, 47, 46, 48, 49, 51, 49, 52, 50, 51, 50, 52, 53, 55, 53, 56, 54, 55, 54, 56, 57, 59, 57, 60, 58, 59, 58, 60, 61, 63, 61, 64, 62, 63, 62, 64, 65, 67, 65, 68, 66, 67, 66, 68, 69, 71, 69, 72, 70, 71, 70, 72, 73, 75, 73, 76, 74, 75, 74, 76, 77, 79, 77, 80, 78, 79, 78, 80, 81, 83, 81, 84, 82, 83, 82, 84, 85, 87, 85, 88, 86, 87, 86, 88, 89, 91, 89, 92, 90, 91, 90, 92, 93, 95, 93, 96, 94, 95, 94, 96, 97, 99, 97, 100, 98, 99, 98, 100, 101, 103, 101, 104, 102, 103, 102, 104, 105, 107, 105, 108, 106, 107, 106, 108, 109, 111, 109, 112, 110, 111, 110, 112, 113, 115, 113, 116, 114, 115, 114, 116, 117, 119, 117, 120, 118, 119, 118, 120, 121, 123, 121, 124, 122, 123, 122, 124, 125, 127, 125, 128, 126, 127, 126, 128, 129, 131, 129, 132, 130, 131, 130, 132, 133, 135, 133, 136, 134, 135, 134, 136, 137, 139, 137, 140, 138, 139, 138, 140, 141, 143, 141, 144, 142, 143, 142, 144, 145, 147, 145, 148, 146, 147, 146, 148, 149, 151, 149, 152, 150, 151, 150, 152, 153, 155, 153, 156, 154, 155, 154, 156, 157, 159, 157, 160, 158, 159, 158, 160, 161, 163, 161, 164, 162, 163, 162, 164, 165, 167, 165, 168, 166, 167, 166, 168, 169, 171, 169, 172, 170, 171, 170, 172, 173, 175, 173, 176, 174, 175, 174, 176, 177, 179, 177, 180, 178, 179, 178, 180, 181, 183, 181, 184, 182, 183, 182, 184, 185, 187, 185, 188, 186, 187, 186, 188, 189, 191, 189, 192, 190, 191, 190, 192, 193, 195, 193, 196, 194, 195, 194, 196, 197, 199, 197, 200, 198, 199, 198, 200, 201, 203, 201, 204, 202, 203, 202, 204, 205, 207, 205, 208, 206, 207, 206, 208, 209, 211, 209, 212, 210, 211, 210, 212, 213, 215, 213, 216, 214, 215, 214, 216, 217, 219, 217, 220, 218, 219, 218, 220, 221, 223, 221, 224, 222, 223, 222, 224, 225, 227, 225, 228, 226, 227, 226, 228, 229, 231, 229, 232, 230, 231, 230, 232, 233, 235, 233, 236, 234, 235, 234, 236, 237, 239, 237, 240, 238, 239, 238, 240, 241, 243, 241, 244, 242, 243, 242, 244, 245, 247, 245, 248, 246, 247, 246, 248, 249, 251, 249, 252, 250, 251, 250, 252, 253, 255, 253, 256, 254, 255, 254, 256, 257, 259, 257, 260, 258, 259, 258, 260, 261, 263, 261, 264, 262, 263, 262, 264, 265, 267, 265, 268, 266, 267, 266, 268, 269, 271, 269, 272, 270, 271, 270, 272, 273, 275, 273, 276, 274, 275, 274, 276, 277, 279, 277, 280, 278, 279, 278, 280, 281, 283, 281, 284, 282, 283, 282, 284, 285, 287, 285, 288, 286, 287, 286, 288, 289, 291, 289, 292, 290, 291, 290, 292, 293, 295, 293, 296, 294, 295, 294, 296, 297, 299, 297, 300, 298, 299, 298, 300, 301, 303, 301, 304, 302, 303, 302, 304, 305, 307, 305, 308, 306, 307, 306, 308, 309, 311, 309, 312, 310, 311, 310, 312, 313, 315, 313, 316, 314, 315, 314, 316, 317, 319, 317, 320, 318, 319, 318, 320, 321, 323, 321, 324, 322, 323, 322, 324, 325, 327, 325, 328, 326, 327, 326, 328, 329, 331, 329, 332, 330, 331, 330, 332, 333, 335, 333, 336, 334, 335, 334, 336, 337, 339, 337, 340, 338, 339, 338, 340, 341, 343, 341, 344, 342, 343, 342, 344, 345, 347, 345, 348, 346, 347, 346, 348, 349, 351, 349, 352, 350, 351, 350, 352, 353, 355, 353, 356, 354, 355, 354, 356, 357, 359, 357, 360, 358, 359, 358, 360, 361, 363, 361, 364, 362, 363, 362, 364, 365, 367, 365, 368, 366, 367, 366, 368, 369, 371, 369, 372, 370, 371, 370, 372, 373, 375, 373, 376, 374, 375, 374, 376, 377, 379, 377, 380, 378, 379, 378, 380, 381, 383, 381, 384, 382, 383, 382, 384, 385, 387, 385, 388, 386, 387, 386, 388, 389, 391, 389, 392, 390, 391, 390, 392, 393, 395, 393, 396, 394, 395, 394, 396, 397, 399, 397, 400, 398, 399, 398, 400, 401, 403, 401, 404, 402, 403, 402, 404, 405, 407, 405, 408, 406, 407, 406, 408, 409, 411, 409, 412, 410, 411, 410, 412, 413, 415, 413, 416, 414, 415, 414, 416, 417, 419, 417, 420, 418, 419, 418, 420, 421, 423, 421, 424, 422, 423, 422, 424, 425, 427, 425, 428, 426, 427, 426, 428, 429, 431, 429, 432, 430, 431, 430, 432], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 96, 97, 97, 98, 98, 99, 99, 100, 100, 101, 101, 102, 102, 103, 103, 104, 104, 105, 105, 106, 106, 107, 107, 108, 108, 109, 109, 110, 110, 111, 111, 112, 112, 113, 113, 114, 114, 115, 115, 116, 116, 117, 117, 118, 118, 119, 119, 120, 120, 121, 121, 122, 122, 123, 123, 124, 124, 125, 125, 126, 126, 127, 127, 128, 128, 129, 129, 130, 130, 131, 131, 132, 132, 133, 133, 134, 134, 135, 135, 136, 136, 137, 137, 138, 138, 139, 139, 140, 140, 141, 141, 142, 142, 143, 143, 144, 144, 145, 145, 146, 146, 147, 147, 148, 148, 149, 149, 150, 150, 151, 151, 152, 152, 153, 153, 154, 154, 155, 155, 156, 156, 157, 157, 158, 158, 159, 159, 160, 160, 161, 161, 162, 162, 163, 163, 164, 164, 165, 165, 166, 166, 167, 167, 168, 168, 169, 169, 170, 170, 171, 171, 172, 172, 173, 173, 174, 174, 175, 175, 176, 176, 177, 177, 178, 178, 179, 179, 180, 180, 181, 181, 182, 182, 183, 183, 184, 184, 185, 185, 186, 186, 187, 187, 188, 188, 189, 189, 190, 190, 191, 191, 192, 192, 193, 193, 194, 194, 195, 195, 196, 196, 197, 197, 198, 198, 199, 199, 200, 200, 201, 201, 202, 202, 203, 203, 204, 204, 205, 205, 206, 206, 207, 207, 208, 208, 209, 209, 210, 210, 211, 211, 212, 212, 213, 213, 214, 214, 215, 215, 216, 216, 217, 217, 218, 218, 219, 219, 220, 220, 221, 221, 222, 222, 223, 223, 224, 224, 225, 225, 226, 226, 227, 227, 228, 228, 229, 229, 230, 230, 231, 231, 232, 232, 233, 233, 234, 234, 235, 235, 236, 236, 237, 237, 238, 238, 239, 239, 240, 240, 241, 241, 242, 242, 243, 243, 244, 244, 245, 245, 246, 246, 247, 247, 248, 248, 249, 249, 250, 250, 251, 251, 252, 252, 253, 253, 254, 254, 255, 255, 256, 256, 257, 257, 258, 258, 259, 259, 260, 260, 261, 261, 262, 262, 263, 263, 264, 264, 265, 265, 266, 266, 267, 267, 268, 268, 269, 269, 270, 270, 271, 271, 272, 272, 273, 273, 274, 274, 275, 275, 276, 276, 277, 277, 278, 278, 279, 279, 280, 280, 281, 281, 282, 282, 283, 283, 284, 284, 285, 285, 286, 286, 287, 287, 288, 288, 289, 289, 290, 290, 291, 291, 292, 292, 293, 293, 294, 294, 295, 295, 296, 296, 297, 297, 298, 298, 299, 299, 300, 300, 301, 301, 302, 302, 303, 303, 304, 304, 305, 305, 306, 306, 307, 307, 308, 308, 309, 309, 310, 310, 311, 311, 312, 312, 313, 313, 314, 314, 315, 315, 316, 316, 317, 317, 318, 318, 319, 319, 320, 320, 321, 321, 322, 322, 323, 323, 324, 324, 325, 325, 326, 326, 327, 327, 328, 328, 329, 329, 330, 330, 331, 331, 332, 332, 333, 333, 334, 334, 335, 335, 336, 336, 337, 337, 338, 338, 339, 339, 340, 340, 341, 341, 342, 342, 343, 343, 344, 344, 345, 345, 346, 346, 347, 347, 348, 348, 349, 349, 350, 350, 351, 351, 352, 352, 353, 353, 354, 354, 355, 355, 356, 356, 357, 357, 358, 358, 359, 359, 360, 360, 361, 361, 362, 362, 363, 363, 364, 364, 365, 365, 366, 366, 367, 367, 368, 368, 369, 369, 370, 370, 371, 371, 372, 372, 373, 373, 374, 374, 375, 375, 376, 376, 377, 377, 378, 378, 379, 379, 380, 380, 381, 381, 382, 382, 383, 383, 384, 384, 385, 385, 386, 386, 387, 387, 388, 388, 389, 389, 390, 390, 391, 391, 392, 392, 393, 393, 394, 394, 395, 395, 396, 396, 397, 397, 398, 398, 399, 399, 400, 400, 401, 401, 402, 402, 403, 403, 404, 404, 405, 405, 406, 406, 407, 407, 408, 408, 409, 409, 410, 410, 411, 411, 412, 412, 413, 413, 414, 414, 415, 415, 416, 416, 417, 417, 418, 418, 419, 419, 420, 420, 421, 421, 422, 422, 423, 423, 424, 424, 425, 425, 426, 426, 427, 427, 428, 428, 429, 429, 430, 430, 431, 431, 432, 432], Int8[-1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1])...);

Delta_1 = SparseArrays.sparse(([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 38, 38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 44, 44, 44, 44, 45, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 47, 48, 48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 50, 51, 51, 51, 51, 52, 52, 52, 52, 53, 53, 53, 53, 54, 54, 54, 54, 55, 55, 55, 55, 56, 56, 56, 56, 57, 57, 57, 57, 58, 58, 58, 58, 59, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 61, 62, 62, 62, 62, 63, 63, 63, 63, 64, 64, 64, 64, 65, 65, 65, 65, 66, 66, 66, 66, 67, 67, 67, 67, 68, 68, 68, 68, 69, 69, 69, 69, 70, 70, 70, 70, 71, 71, 71, 71, 72, 72, 72, 72, 73, 73, 73, 73, 74, 74, 74, 74, 75, 75, 75, 75, 76, 76, 76, 76, 77, 77, 77, 77, 78, 78, 78, 78, 79, 79, 79, 79, 80, 80, 80, 80, 81, 81, 81, 81, 82, 82, 82, 82, 83, 83, 83, 83, 84, 84, 84, 84, 85, 85, 85, 85, 86, 86, 86, 86, 87, 87, 87, 87, 88, 88, 88, 88, 89, 89, 89, 89, 90, 90, 90, 90, 91, 91, 91, 91, 92, 92, 92, 92, 93, 93, 93, 93, 94, 94, 94, 94, 95, 95, 95, 95, 96, 96, 96, 96, 97, 97, 97, 97, 98, 98, 98, 98, 99, 99, 99, 99, 100, 100, 100, 100, 101, 101, 101, 101, 102, 102, 102, 102, 103, 103, 103, 103, 104, 104, 104, 104, 105, 105, 105, 105, 106, 106, 106, 106, 107, 107, 107, 107, 108, 108, 108, 108], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432], Int8[1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1])...);

T = [Delta_0, Delta_1];

In [33]:
@benchmark LC.chainCongruenceSM(G, T)

BenchmarkTools.Trial: 934 samples with 1 evaluation.
 Range (min … max):  4.852 ms … 16.615 ms  ┊ GC (min … max): 0.00% … 67.28%
 Time  (median):     5.033 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.346 ms ±  1.133 ms  ┊ GC (mean ± σ):  2.89% ±  8.44%

  █▆▅▅▄▂                                                      
  ████████▇██▆██▇▇▆▅▆▄▄▁▁▅▁▄▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▆▆ █
  4.85 ms      Histogram: log(frequency) by time     11.5 ms <

 Memory estimate: 1.21 MiB, allocs estimate: 18236.

## ESEMPIO 2: GraphBLAS based

In [34]:
Delta_0_GB = SparseMM.sm2gbm(Delta_0)
Delta_1_GB = SparseMM.sm2gbm(Delta_1)
T_GB = [Delta_0_GB, Delta_1_GB]

2-element Vector{GrB_Matrix{Int8}}:
 
 

GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}GrB_Matrix{Int8}

In [35]:
@benchmark LC.chainCongruenceGB(W, T_GB)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  326.265 μs …   4.046 ms  ┊ GC (min … max): 0.00% … 74.74%
 Time  (median):     341.068 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   350.962 μs ± 150.043 μs  ┊ GC (mean ± σ):  1.49% ±  3.19%

      ▅▇▇▆█▅▂                                                    
  ▂▄▆████████▇▇▇▆▆▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂ ▃
  326 μs           Histogram: frequency by time          434 μs <

 Memory estimate: 101.58 KiB, allocs estimate: 477.